In [1]:
# ONLY FOR GOOGLE COLAB
from google.colab import drive
import os
drive.mount('/content/drive')
WORKING_DIRECTORY = '/content/drive/My Drive/gazprom'

os.chdir(WORKING_DIRECTORY)

Mounted at /content/drive


In [2]:
!pip install catboost
import pandas as pd
import catboost as ct
import numpy as np

     |████████████████████████████████| 66.2MB 65kB/s 


In [3]:
transactions = pd.read_csv('tr_months.csv').drop(['Unnamed: 0'], axis=1)

In [4]:
transactions = transactions[transactions['last_day_month'] < 20]

In [5]:
a = pd.unique(transactions['mcc_group'])
num_month = pd.unique(transactions['last_day_month']).shape[0]

datadata = {}
for i in a:
    ttt = transactions[transactions['mcc_group'] == i].drop('mcc_group', axis=1)

    cat_agg = pd.pivot(ttt, index='client_id', columns='last_day_month')
    if cat_agg.shape[1] != 5 * num_month:
        continue
    cat_agg[cat_agg.isna() == True] = 0

    # Shift
    for it, row in cat_agg.iterrows():
        sh = np.flatnonzero(row)
        for t in ['att_sum', 'att_cnt', 'att_sum_min', 'att_sum_max', 'att_sum_avg']:
            cat_agg.loc[it].at[t] = cat_agg.loc[it].at[t].shift(-sh[0], fill_value=0)
    
    cat_agg.columns = [str(group) + '-' + str(agg) for agg, group in cat_agg.columns]
    cat_agg[cat_agg.columns[:num_month]] = cat_agg[cat_agg.columns[:num_month]].cumsum(axis=1)
    cat_agg[cat_agg.columns[num_month:2*num_month]] = cat_agg[cat_agg.columns[num_month:2*num_month]].cumsum(axis=1)

    divisor = cat_agg[cat_agg.columns[:num_month]]
    divisor.columns = cat_agg.columns[4*num_month:]

    dividend = cat_agg[cat_agg.columns[num_month:2*num_month]]
    dividend.columns = cat_agg.columns[4*num_month:]
    cat_agg[cat_agg.columns[4*num_month:]] = divisor / dividend
    datadata[i] = cat_agg

In [6]:
datadata['TRANSPORTATION']

,0-att_sum,1-att_sum,2-att_sum,3-att_sum,4-att_sum,5-att_sum,6-att_sum,7-att_sum,8-att_sum,9-att_sum,10-att_sum,11-att_sum,12-att_sum,13-att_sum,14-att_sum,15-att_sum,16-att_sum,17-att_sum,18-att_sum,19-att_sum,0-att_cnt,1-att_cnt,2-att_cnt,3-att_cnt,4-att_cnt,5-att_cnt,6-att_cnt,7-att_cnt,8-att_cnt,9-att_cnt,10-att_cnt,11-att_cnt,12-att_cnt,13-att_cnt,14-att_cnt,15-att_cnt,16-att_cnt,17-att_cnt,18-att_cnt,19-att_cnt,...,0-att_sum_max,1-att_sum_max,2-att_sum_max,3-att_sum_max,4-att_sum_max,5-att_sum_max,6-att_sum_max,7-att_sum_max,8-att_sum_max,9-att_sum_max,10-att_sum_max,11-att_sum_max,12-att_sum_max,13-att_sum_max,14-att_sum_max,15-att_sum_max,16-att_sum_max,17-att_sum_max,18-att_sum_max,19-att_sum_max,0-att_sum_avg,1-att_sum_avg,2-att_sum_avg,3-att_sum_avg,4-att_sum_avg,5-att_sum_avg,6-att_sum_avg,7-att_sum_avg,8-att_sum_avg,9-att_sum_avg,10-att_sum_avg,11-att_sum_avg,12-att_sum_avg,13-att_sum_avg,14-att_sum_avg,15-att_sum_avg,16-att_sum_avg,17-att_sum_avg,18-att_sum_avg,19-att_sum_avg
client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4829.00,4829.00,4829.00,5529.00,5529.00,5529.00,5529.00,5529.00,5529.00,5529.00,5529.00,5529.00,5529.00,5529.00,5529.00,5529.00,5529.00,5529.00,5529.00,5529.00,23.0,23.0,23.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,24.0,...,456.00,0.0,0.0,700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,209.956522,209.956522,209.956522,230.375000,230.375,230.375,230.375,230.375,230.375000,230.375000,230.375000,230.375000,230.375000,230.375000,230.375000,230.375000,230.375000,230.375000,230.375000,230.375000
2,107.00,107.00,107.00,107.00,107.00,107.00,107.00,107.00,107.00,107.00,107.00,107.00,107.00,107.00,107.00,107.00,107.00,107.00,107.00,107.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,107.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,107.000000,107.000000,107.000000,107.000000,107.000,107.000,107.000,107.000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000,107.000000
12,294.00,588.00,1029.00,1176.00,1323.00,1617.00,2058.00,2499.00,2499.00,2499.00,2539.00,2539.00,2539.00,2539.00,2539.00,2539.00,2539.00,2539.00,2539.00,2539.00,2.0,4.0,7.0,8.0,9.0,11.0,14.0,17.0,17.0,17.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,...,147.00,147.0,147.0,147.0,147.0,147.0,147.0,147.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,147.000000,147.000000,147.000000,147.000000,147.000,147.000,147.000,147.000,147.000000,147.000000,133.631579,133.631579,133.631579,133.631579,133.631579,133.631579,133.631579,133.631579,133.631579,133.631579
14,613.20,613.20,613.20,613.20,613.20,613.20,613.20,613.20,613.20,613.20,613.20,613.20,613.20,613.20,613.20,613.20,613.20,613.20,613.20,613.20,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,613.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,613.200000,613.200000,613.200000,613.200000,613.200,613.200,613.200,613.200,613.200000,613.200000,613.200000,613.200000,613.200000,613.200000,613.200000,613.200000,613.200000,613.200000,613.200000,613.200000
16,1403.70,1403.70,1403.70,1403.70,1403.70,1403.70,1403.70,1403.70,1403.70,1403.70,1403.70,1403.70,1403.70,1403.70,1403.70,1403.70,1403.70,1403.70,1403.70,1403.70,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1403.70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1403.700000,1403.700000,1403.700000,1403.700000,1403.700,1403.700,1403.700,1403.700,1403.700000,1403.700000,1403.700000,1403.700000,1403.700000,1403.700000,1403.700000,1403.700000,1403.700000,1403.700000,1403.700000,1403.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [7]:
template = ['att_sum', 'att_cnt', 'att_sum_min', 'att_sum_max', 'att_sum_avg']

In [8]:
N = 7
columns = [str(u) + '-' + e for e in template for u in range(N)] + [str(N) + '-' + template[0]]

In [9]:
data_end = {}
for e in datadata.keys():
    try:
        df = pd.DataFrame(columns=columns)
        for i in range(0, num_month-N):
            new_cols = []
            for c in columns:
                ind = int(c.split('-')[0])
                new_cols.append(str(i+ind) + '-' + c.split('-')[1])
            d = datadata[e][new_cols]
            d.columns = columns
            df = df.append(d)
        
        data_end[e] = df
    except KeyError:
        pass


In [19]:
data_end['CONSTRUCTION SERVICES']

,0-att_sum,1-att_sum,2-att_sum,3-att_sum,4-att_sum,5-att_sum,6-att_sum,0-att_cnt,1-att_cnt,2-att_cnt,3-att_cnt,4-att_cnt,5-att_cnt,6-att_cnt,0-att_sum_min,1-att_sum_min,2-att_sum_min,3-att_sum_min,4-att_sum_min,5-att_sum_min,6-att_sum_min,0-att_sum_max,1-att_sum_max,2-att_sum_max,3-att_sum_max,4-att_sum_max,5-att_sum_max,6-att_sum_max,0-att_sum_avg,1-att_sum_avg,2-att_sum_avg,3-att_sum_avg,4-att_sum_avg,5-att_sum_avg,6-att_sum_avg,7-att_sum
34,370.00,370.00,370.00,370.00,370.00,370.00,370.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,370.00,0.0,0.0,0.0,0.0,0.0,0.0,370.00,0.0,0.0,0.0,0.0,0.0,0.0,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.00
104,240.00,240.00,240.00,240.00,240.00,240.00,240.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,240.00,0.0,0.0,0.0,0.0,0.0,0.0,240.00,0.0,0.0,0.0,0.0,0.0,0.0,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,240.00
346,12570.00,12570.00,12570.00,12570.00,12570.00,12570.00,12570.00,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4670.00,0.0,0.0,0.0,0.0,0.0,0.0,7900.00,0.0,0.0,0.0,0.0,0.0,0.0,6285.000000,6285.000000,6285.000000,6285.000000,6285.000000,6285.000000,6285.000000,12570.00
498,130.00,130.00,130.00,130.00,130.00,130.00,130.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,130.00,0.0,0.0,0.0,0.0,0.0,0.0,130.00,0.0,0.0,0.0,0.0,0.0,0.0,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.00
568,406.79,406.79,406.79,406.79,406.79,406.79,406.79,1.0,1.0,1.0,1.0,1.0,1.0,1.0,406.79,0.0,0.0,0.0,0.0,0.0,0.0,406.79,0.0,0.0,0.0,0.0,0.0,0.0,406.790000,406.790000,406.790000,406.790000,406.790000,406.790000,406.790000,406.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9876,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.00
9898,700.00,700.00,700.00,700.00,700.00,700.00,700.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.00
9910,3697.00,3697.00,3697.00,3697.00,3697.00,3697.00,3697.00,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1232.333333,1232.333333,1232.333333,1232.333333,1232.333333,1232.333333,1232.333333,3697.00
9935,223.70,223.70,223.70,223.70,223.70,223.70,223.70,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,223.700000,223.700000,223.700000,223.700000,223.700000,223.700000,223.700000,223.70


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
data_fit = {}
for e in data_end.keys():
    s = 0
    cat = ct.CatBoostRegressor(task_type="GPU", iterations=2000, depth=4)

    X = data_end[e].drop(str(N) + '-att_sum', axis=1)
    y = data_end[e][str(N) + '-att_sum']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    y_test = y_test.reset_index().drop(['index'], axis=1)

    X_train = X_train.reset_index().drop(['index'], axis=1)
    y_train = y_train.reset_index().drop(['index'], axis=1)
    X_train1 = X_train
    
    cat.fit(X_train, y_train, verbose=False, plot=True)
    predict = cat.predict(X_test)
    for i in range(len(predict)):
        s += abs(predict[i] - y_test.at[i, str(N) + '-att_sum'])
    print(predict, y_test)
    print(e, s/sum(y_test[str(N) + '-att_sum']), mean_absolute_error(predict, y_test), mean_squared_error(predict, y_test))
    data_fit[e] = cat


In [21]:
import sklearn.linear_model

In [22]:
data_fit = {}
for e in data_end.keys():
    s, s1 = 0, 0
    cat = sklearn.linear_model.LinearRegression()

    X = data_end[e].drop(str(N) + '-att_sum', axis=1)
    y = data_end[e][str(N) + '-att_sum']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    y_test = y_test.reset_index().drop(['index'], axis=1)

    X_train = X_train.reset_index().drop(['index'], axis=1)
    y_train = y_train.reset_index().drop(['index'], axis=1)
    X_train1 = X_train
    
    cat.fit(X_train, y_train)
    predict = cat.predict(X_test)
    for i in range(len(predict)):
        s1 += abs(predict[i] - y_test.at[i, str(N) + '-att_sum'])
        s += abs(predict[i] - y_test.at[i, str(N) + '-att_sum'])/(y_test.at[i, str(N) + '-att_sum'])
    print(predict, y_test)
    print(e, s/len(predict), s1/sum(y_test[str(N) + '-att_sum']), mean_absolute_error(predict, y_test), mean_squared_error(predict, y_test))
    data_fit[e] = cat

[[28389.56823301]
 [ 2535.59579961]
 [33046.84405817]
 ...
 [49436.62872539]
 [  318.42985436]
 [14797.80256746]]        7-att_sum
0       27941.40
1        1915.00
2       30443.18
3         160.00
4        5404.20
...          ...
15504   17633.00
15505    4504.00
15506   46672.00
15507      21.00
15508   14125.00

[15509 rows x 1 columns]
TRANSPORTATION [0.78397318] [0.07573902] 1919.252920219737 107465363.76564333
[[64646.26161432]
 [16197.17281287]
 [10109.04310227]
 ...
 [56115.44798434]
 [19950.75114304]
 [76571.66262253]]        7-att_sum
0       61801.54
1       12485.07
2       19204.95
3       72169.46
4       12955.20
...          ...
30420  214734.67
30421     820.00
30422   58936.76
30423   18913.41
30424   71905.92

[30425 rows x 1 columns]
RETAIL STORES [0.12275125] [0.0486658] 3435.8225252851903 65009502.932537645
[[442806.1346602 ]
 [854859.42341366]
 [630344.74355433]
 ...
 [808659.48133384]
 [242709.2852318 ]
 [242849.89851564]]        7-att_sum
0      470900.00
1  

In [23]:
data_fit['RETAIL STORES'].save_model('lin.pkl')

AttributeError: ignored

In [24]:
from joblib import dump, load

In [25]:
dump(data_fit['RETAIL STORES'], 'lin.pkl')

['lin.pkl']